Выполнила Теплякова Юлия
Группа 6403-010302

## Подключение библиотек

In [1]:
import numpy as np

# **Часть 1**

## Сформировать порождающую матрицу линейного кода (7, 4, 3)

In [2]:
# Размерности кода
k = 4
n = 7
d = 3

# 𝐺 = [𝐼𝑘|𝑋]
# Создание единичной матрицы 𝐼𝑘
I_k = np.eye(k, dtype=int)

# Функция для генерации матрицы X
def generate_X(k, n):
    rows = []
    total_columns = n - k

    # Генерация всех возможных двоичных строк длины total_columns
    for i in range(1 << total_columns):
      row = [int(bit) for bit in bin(i)[2:].zfill(total_columns)]

      if sum(row) >= 2 and row not in rows:
        rows.append(row)

      if len(rows) >= k:
        break

    return np.array(rows, dtype=int)

X = generate_X(k, n)

# Формирование порождающей матрицы G
G = np.hstack((I_k, X))

print("Порождающая матрица G:")
print(G)

Порождающая матрица G:
[[1 0 0 0 0 1 1]
 [0 1 0 0 1 0 1]
 [0 0 1 0 1 1 0]
 [0 0 0 1 1 1 1]]


## Сформировать проверочную матрицу на основе порождающей

In [3]:
# Создани единичной матрицы I_{n-k}
I_n_k = np.eye(n-k, dtype=int)

# 𝐻 = [ 𝑋
#      𝐼𝑛−𝑘]
H = np.vstack((X, I_n_k))

print("Проверочная матрица H:")
print(H)

Проверочная матрица H:
[[0 1 1]
 [1 0 1]
 [1 1 0]
 [1 1 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]]


## Сформировать таблицу синдромов для всех однократных ошибок

In [4]:
# Создание таблицы синдромов
syndromes = {}
for i in range(n):
    E = np.zeros(n, dtype=int)
    E[i] = 1

    # Вычисление синдрома
    S = (E @ H) % 2

    # Сохранение синдрома в таблицу
    syndromes[tuple(S)] = tuple(E)

print("Таблица синдромов:")
for syndrome, error_vector in syndromes.items():
  print(f"Синдром {syndrome} -> Ошибка {error_vector}")

Таблица синдромов:
Синдром (0, 1, 1) -> Ошибка (1, 0, 0, 0, 0, 0, 0)
Синдром (1, 0, 1) -> Ошибка (0, 1, 0, 0, 0, 0, 0)
Синдром (1, 1, 0) -> Ошибка (0, 0, 1, 0, 0, 0, 0)
Синдром (1, 1, 1) -> Ошибка (0, 0, 0, 1, 0, 0, 0)
Синдром (1, 0, 0) -> Ошибка (0, 0, 0, 0, 1, 0, 0)
Синдром (0, 1, 0) -> Ошибка (0, 0, 0, 0, 0, 1, 0)
Синдром (0, 0, 1) -> Ошибка (0, 0, 0, 0, 0, 0, 1)


## Сформировать кодовое слово длины n из слова длины k. Внести однократную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова.

In [5]:
# Формирование кодового слова (n) из случайного исходного слова (k)
message = np.random.randint(0, 2, size=k)
print("Исходное сообщение:", message)
codeword = (message @ G) % 2
print("Отправлено:", codeword)

# Внесение однократной ошибки в кодовое слово
error_position = np.random.randint(0, n)
codeword_with_error = codeword.copy()
codeword_with_error[error_position] ^= 1
print("В ходе передачи возникла ошибка\nПринято:", codeword_with_error)

# Вычисление синдрома
syndrome = (codeword_with_error @ H) % 2
print("Синдром:", syndrome)

# Исправление ошибки
if tuple(syndrome) in syndromes:
    error_vector = syndromes[tuple(syndrome)]
    print("В принятом сообщении есть ошибка\nОшибка:", error_vector)

    corrected_codeword = codeword_with_error.copy()
    corrected_codeword[np.where(np.array(error_vector) == 1)[0]] ^= 1

    # Проверка на отличие от отправленного
    if not np.array_equal(corrected_codeword, codeword):
        print("Исправленное слово отличается от отправленного.")
    else:
        print("Исправленное слово совпадает с отправленным.")
else:
    corrected_codeword = codeword_with_error
print("Исправленное сообщение:", corrected_codeword)

# Декодирование исправленного кодового слова
decoded_message = (corrected_codeword @ G.T) % 2
print("Декодированное сообщение:", decoded_message)

Исходное сообщение: [0 1 0 0]
Отправлено: [0 1 0 0 1 0 1]
В ходе передачи возникла ошибка
Принято: [0 1 0 1 1 0 1]
Синдром: [1 1 1]
В принятом сообщении есть ошибка
Ошибка: (0, 0, 0, 1, 0, 0, 0)
Исправленное слово совпадает с отправленным.
Исправленное сообщение: [0 1 0 0 1 0 1]
Декодированное сообщение: [1 1 1 0]


## Сформировать кодовое слово длины n из слова длины k. Внести двукратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться, что полученное слово отличается от отправленного

In [6]:
# Формирование кодового слова (n) из случайного исходного слова (k)
message = np.random.randint(0, 2, size=k)
print("Исходное сообщение:", message)
codeword = (message @ G) % 2
print("Отправлено:", codeword)

# Внесение двукратной ошибки в кодовое слово
error_positions = np.random.choice(n, 2, replace=False)
error_codeword = codeword.copy()
error_codeword[error_positions] = (error_codeword[error_positions] + 1) % 2
print("В ходе передачи возникла ошибка\nПринято:", error_codeword)

# Вычисление синдрома
syndrome = (error_codeword @ H) % 2
print("Синдром:", syndrome)

# Попытка исправить ошибку
error_vector = syndromes.get(tuple(syndrome), None)
if error_vector is not None:
    print("В принятом сообщении есть ошибка\nОшибка:", error_vector)
    corrected_codeword = error_codeword.copy()
    corrected_codeword[np.where(error_vector)[0]] ^= 1
    print("Исправленное сообщение:", corrected_codeword)

    # Проверка на отличие от отправленного
    if not np.array_equal(corrected_codeword, codeword):
        print("Исправленное слово отличается от отправленного.")
    else:
        print("Исправленное слово совпадает с отправленным.")
else:
    print("Ошибка не может быть исправлена.")

Исходное сообщение: [1 1 1 0]
Отправлено: [1 1 1 0 0 0 0]
В ходе передачи возникла ошибка
Принято: [0 1 1 1 0 0 0]
Синдром: [1 0 0]
В принятом сообщении есть ошибка
Ошибка: (0, 0, 0, 0, 1, 0, 0)
Исправленное сообщение: [0 1 1 1 1 0 0]
Исправленное слово отличается от отправленного.


## **Часть 2**


## Сформировать порождающую матрицу линейного кода (n, k, 5)

In [16]:
# а) было не меньше 4 единиц в каждой строке;
# б) сумма любых двух строк содержала не менее 3 единиц;
# в) сумма любых трёх строк содержала не менее 2 единиц;
# г) сумма любых четырёх строк содержала не менее 1 единицы;

def check_conditions(X):
    # Проверка условия (а)
    for row in X:
        if np.sum(row) < 4:
            return False

    # Проверка условия (б)
    for i in range(len(X)):
        for j in range(i + 1, len(X)):
            if np.sum(X[i] + X[j]) < 3:
                return False

    # Проверка условия (в)
    for i in range(len(X)):
        for j in range(i + 1, len(X)):
            for k in range(j + 1, len(X)):
                if np.sum(X[i] + X[j] + X[k]) < 2:
                    return False

    # Проверка условия (г)
    for i in range(len(X)):
        for j in range(i + 1, len(X)):
            for k in range(j + 1, len(X)):
                for l in range(k + 1, len(X)):
                    if np.sum(X[i] + X[j] + X[k] + X[l]) < 1:
                        return False

    return True

def generate_generator_matrix(n, k):
    # Инициализация порождающей матрицы
    G = np.zeros((k, n), dtype=int)

    # Заполнение единичной матрицы
    G[:, :k] = np.eye(k, dtype=int)

    # Генерация матрицы X
    found = False
    while not found:
        X = np.random.randint(0, 2, size=(k, n - k))
        G[:, k:] = X

        if check_conditions(X):
            found = True

    return G

# Параметры
n = 7
k = 3

# Генерация порождающей матрицы
G = generate_generator_matrix(n, k)

print("Порождающая матрица G:")
print(G)

Порождающая матрица G:
[[1 0 0 1 1 1 1]
 [0 1 0 1 1 1 1]
 [0 0 1 1 1 1 1]]


## Сформировать проверочную матрицу на основе порождающей.

In [17]:
def generate_parity_check_matrix(G):
  X = generate_X(k, n)
  I_n_k = np.eye(n-k, dtype=int)
  H = np.vstack((X, I_n_k))

  return H

H = generate_parity_check_matrix(G)

print("Проверочная матрица H:")
print(H)

Проверочная матрица H:
[[0 0 1 1]
 [0 1 0 1]
 [0 1 1 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]


## Сформировать таблицу синдромов для всех однократных и двукратных ошибок

In [18]:
def generate_syndrome_table(H):
    syndrome_table = {}

    # Однократные ошибки
    for i in range(n):
        e = np.zeros(n, dtype=int)
        e[i] = 1  # Ошибка в позиции i
        syndrome = np.dot(e, H) % 2
        syndrome_table[tuple(syndrome)] = f"Ошибка на позиции {i + 1}"

    # Двукратные ошибки
    for i in range(n):
        for j in range(i + 1, n):
            e = np.zeros(n, dtype=int)
            e[i] = 1
            e[j] = 1  # Ошибка в позициях i и j
            syndrome = np.dot(e, H) % 2
            syndrome_table[tuple(syndrome)] = f"Ошибка на позиции ({i + 1}, {j + 1})"

    return syndrome_table

syndrome_table = generate_syndrome_table(H)

print("Таблица синдромов:")
for syndrome, error in syndrome_table.items():
  print(f"Синдром {syndrome} -> Ошибка {error}")

Таблица синдромов:
Синдром (0, 0, 1, 1) -> Ошибка Ошибка на позиции (6, 7)
Синдром (0, 1, 0, 1) -> Ошибка Ошибка на позиции (5, 7)
Синдром (0, 1, 1, 0) -> Ошибка Ошибка на позиции (5, 6)
Синдром (1, 0, 0, 0) -> Ошибка Ошибка на позиции 4
Синдром (0, 1, 0, 0) -> Ошибка Ошибка на позиции (3, 6)
Синдром (0, 0, 1, 0) -> Ошибка Ошибка на позиции (3, 5)
Синдром (0, 0, 0, 1) -> Ошибка Ошибка на позиции (2, 5)
Синдром (1, 0, 1, 1) -> Ошибка Ошибка на позиции (1, 4)
Синдром (0, 1, 1, 1) -> Ошибка Ошибка на позиции (3, 7)
Синдром (1, 1, 0, 1) -> Ошибка Ошибка на позиции (2, 4)
Синдром (1, 1, 1, 0) -> Ошибка Ошибка на позиции (3, 4)
Синдром (1, 1, 0, 0) -> Ошибка Ошибка на позиции (4, 5)
Синдром (1, 0, 1, 0) -> Ошибка Ошибка на позиции (4, 6)
Синдром (1, 0, 0, 1) -> Ошибка Ошибка на позиции (4, 7)


## Сформировать кодовое слово длины n из слова длины k. Внести однократную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова.

In [32]:
# Формирование кодового слова (n) из случайного исходного слова (k)
message = np.random.randint(0, 2, size=k)
print("Исходное сообщение:", message)
codeword = (message @ G) % 2
print("Отправлено:", codeword)

# Внесение двукратной ошибки в кодовое слово
error_positions = np.random.choice(n, 2, replace=False)
error_codeword = codeword.copy()
error_codeword[error_position] ^= 1
print("В ходе передачи возникла ошибка\nПринято:", error_codeword)

# Вычисление синдрома
syndrome = (error_codeword @ H) % 2
print("Синдром:", syndrome)

# Попытка исправить ошибку
error_correction = syndrome_table.get(tuple(syndrome), None)
print(error_correction)

if error_correction:
  corrected_codeword = np.copy(error_codeword)
  corrected_codeword[error_position] ^= 1
  print(f"Исправленное кодовое слово: {corrected_codeword}")
else:
  print("Ошибка не исправлена.")

# Проверка на отличие от отправленного
if not np.array_equal(corrected_codeword, codeword):
  print("Исправленное слово отличается от отправленного.")
else:
  print("Исправленное слово совпадает с отправленным.")

Исходное сообщение: [0 1 1]
Отправлено: [0 1 1 0 0 0 0]
В ходе передачи возникла ошибка
Принято: [0 1 1 0 1 0 0]
Синдром: [0 1 1 1]
Ошибка на позиции (3, 7)
Исправленное кодовое слово: [0 1 1 0 0 0 0]
Исправленное слово совпадает с отправленным.


## Сформировать кодовое слово длины n из слова длины k. Внести двукратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова.

In [42]:
# Формирование кодового слова (n) из случайного исходного слова (k)
message = np.random.randint(0, 2, size=k)
print("Исходное сообщение:", message)
codeword = (message @ G) % 2  # формирование кодового слова
print("Отправлено:", codeword)

# Внесение двукратной ошибки в кодовое слово
error_positions = np.random.choice(n, 2, replace=False)
error_codeword = codeword.copy()
error_codeword[error_positions] ^= 1  # внесение ошибок
print("В ходе передачи возникла ошибка\nПринято:", error_codeword)

# Вычисление синдрома
syndrome = (error_codeword @ H) % 2
print("Синдром:", syndrome)

# Генерация таблицы синдромов
syndrome_table = generate_syndrome_table(H)  # Убедитесь, что эта функция реализована

# Попытка исправить ошибку
error_correction = syndrome_table.get(tuple(syndrome), None)

if error_correction:
    corrected_codeword = np.copy(error_codeword)

    for pos in error_positions:
        corrected_codeword[pos] ^= 1
    print(f"Исправленное кодовое слово: {corrected_codeword}")
else:
    print("Ошибка не исправлена.")

# Проверка на отличие от отправленного
if not np.array_equal(corrected_codeword, codeword):
    print("Исправленное слово отличается от отправленного.")
else:
    print("Исправленное слово совпадает с отправленным.")

Исходное сообщение: [1 0 1]
Отправлено: [1 0 1 0 0 0 0]
В ходе передачи возникла ошибка
Принято: [0 0 1 0 0 1 0]
Синдром: [0 1 0 0]
Исправленное кодовое слово: [1 0 1 0 0 0 0]
Исправленное слово совпадает с отправленным.


## Сформировать кодовое слово длины n из слова длины k. Внести трёхкратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться, что полученное слово отличается от отправленного.

In [43]:
# Формирование кодового слова (n) из случайного исходного слова (k)
message = np.random.randint(0, 2, size=k)
print("Исходное сообщение:", message)
codeword = (message @ G) % 2
print("Отправлено:", codeword)

# Внесение трехкратной ошибки в кодовое слово
error_positions = np.random.choice(n, 3, replace=False)
error_codeword = codeword.copy()
error_codeword[error_positions] ^= 1
print("В ходе передачи возникла ошибка\nПринято:", error_codeword)

# Вычисление синдрома
syndrome = (error_codeword @ H) % 2
print("Синдром:", syndrome)

# Генерация таблицы синдромов
syndrome_table = generate_syndrome_table(H)

# Попытка исправить ошибку
error_correction = syndrome_table.get(tuple(syndrome), None)

if error_correction is not None:
    corrected_codeword = np.copy(error_codeword)

    # Убедимся, что error_correction является списком целых чисел
    if isinstance(error_correction, list):
        for pos in error_correction:
            corrected_codeword[pos] ^= 1
        print(f"Исправленное кодовое слово: {corrected_codeword}")
    else:
        print("Ошибка в формате данных для исправления.")
else:
    print("Ошибка не исправлена.")

# Проверка на отличие от отправленного
if not np.array_equal(corrected_codeword, codeword):
    print("Исправленное слово отличается от отправленного.")
else:
    print("Исправленное слово совпадает с отправленным.")


Исходное сообщение: [1 0 1]
Отправлено: [1 0 1 0 0 0 0]
В ходе передачи возникла ошибка
Принято: [0 1 1 0 1 0 0]
Синдром: [0 1 1 1]
Ошибка в формате данных для исправления.
Исправленное слово отличается от отправленного.
